#### Imports

In [63]:
import grpc
import tensorflow as tf
from keras_image_helper import create_preprocessor
from tensorflow.keras.models import load_model
from tensorflow_serving.apis import prediction_service_pb2_grpc, predict_pb2

In [64]:
host = 'localhost:8502'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [65]:
preprocessor = create_preprocessor('xception', target_size=(32, 32))

In [66]:
url = 'https://c7.alamy.com/comp/K0W4HC/a-traffic-sign-indicating-a-speed-limit-of-50-kmh-seen-near-tubingen-K0W4HC.jpg'
X = preprocessor.from_url(url)

In [67]:
# X

In [68]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [69]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'traffic-sign-recognition-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [70]:
# pb_request

In [71]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8502: Failed to connect to remote host: connect: Connection refused (61)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8502: Failed to connect to remote host: connect: Connection refused (61)", grpc_status:14, created_time:"2025-01-13T17:22:06.919061-05:00"}"
>

In [59]:
pb_response

NameError: name 'pb_response' is not defined

In [60]:
preds = pb_response.outputs['dense_7'].float_val

NameError: name 'pb_response' is not defined

In [61]:
classes = [
'Speed limit (20km/h)',
'Speed limit (30km/h)',
'Speed limit (50km/h)',
'Speed limit (60km/h)',
'Speed limit (70km/h)',
'Speed limit (80km/h)',
'End of speed limit (80km/h)',
'Speed limit (100km/h)',
'Speed limit (120km/h)',
'No passing',
'No passing for vehicles over 3.5 metric tons',
'Right-of-way at the next intersection',
'Priority road',
'Yield',
'Stop',
'No vehicles',
'Vehicles over 3.5 metric tons prohibited',
'No entry',
'General caution',
'Dangerous curve to the left',
'Dangerous curve to the right',
'Double curve',
'Bumpy road',
'Slippery road',
'Road narrows on the right',
'Road work',
'Traffic signals',
'Pedestrians',
'Children crossing',
'Bicycles crossing',
'Beware of ice/snow',
'Wild animals crossing',
'End of all speed and passing limits',
'Turn right ahead',
'Turn left ahead',
'Ahead only',
'Go straight or right',
'Go straight or left',
'Keep right',
'Keep left',
'Roundabout mandatory',
'End of no passing',
'End of no passing by vehicles over 3.5 metric'
]

In [ ]:
dict(zip(classes, preds))